# Aplicación de envío rápido de correos. 📧📧📧📨

### Conexión con la base de datos PostGreSQL

In [8]:
import credenciales
#Librerias e Inicio de variables
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
import sys, os
import datetime as dt

In [10]:
print(credenciales.__file__)


c:\Users\Jreyesm\Desktop\noti_correo\credenciales.py


In [9]:
# CONEXION BASE DE DATOS
conn = psycopg2.connect(
      host=credenciales.HOST,
      port=credenciales.PORT,
      database=credenciales.DATA_BASE,
      user=credenciales.USER,
      password=credenciales.PASSWORD
)


AttributeError: module 'credenciales' has no attribute 'HOST'

In [ ]:
# Consulta SQL
sql4 = '''
WITH DatosOrdenados AS (
  SELECT
    c.idexcel, 
    c.nombreexcel,
    e.nomempresa AS nom_empresa,
    t.sigla_terminal AS nom_terminal,
    s2.idtipo,
    TO_CHAR(s.finicio, 'YYYYMM') AS fechasol,
    c.created_at,
    ROW_NUMBER() OVER(
      PARTITION BY e.nomempresa, t.sigla_terminal, s2.idtipo, TO_CHAR(s.finicio, 'YYYYMM') 
      ORDER BY c.created_at
    ) AS rn
  FROM db_sielp.api.cargaexcel c
  INNER JOIN db_sielp.api.historialsolicitud h ON h.idexcel = c.idexcel
  INNER JOIN db_sielp.api.solempresas s ON s.id = h.idsolicitudempresa
  INNER JOIN db_sielp.api.solicitudes s2 ON s.idsolicitud = s2.id
  INNER JOIN db_sielp.api.empresas e ON e.id = s.idempresa
  INNER JOIN db_sielp.api.terminales t ON t.id_terminal = s.idterminal
  WHERE
   s.finicio  BETWEEN  %(start_date_q4)s AND %(end_date_q4)s
)
select
	d.*,
    p.*
FROM 
    DatosOrdenados d
JOIN 
    db_sielp.api.cargaexceltmp tmp ON tmp.idexcel = d.idexcel
CROSS JOIN LATERAL
jsonb_to_record(tmp.data::jsonb) AS p(
    id text,
    terminal text,
    largo_muelle text, -- Campo adicional único del segundo objeto JSON
    largo_lap text,
    gruas_portico text, -- Campo adicional único del segundo objeto JSON
    enchufes_reefer_fijos text,
    enchufes_reefer_moviles text,
    area_total text,
    area_contenedor text,
    area_fraccionada text,
    area_granel_solido text,
    area_extension text,
    calado_maximo text,
    observaciones text
    )
WHERE 
    d.rn = 1 
    AND d.idtipo in (12,4,18)
    AND p.terminal = %(current_terminal)s
 


'''

dfi1 = pd.read_sql_query(sql4, conn, params=params)
dfi1 = dfi1.iloc[:, 8:]
conn.close()

### Filtros y estructura del mensaje.

### Configuración con correos electrónicos